In [0]:
def mount_adls(storage_account_name, container_name):
    # get secrets from Azure key vault
    clientID = dbutils.secrets.get(scope = 'formula1-datalake-access-key', key = 'azuredatabricks-dl-formula1app-clientid')
    tenantId = dbutils.secrets.get(scope = 'formula1-datalake-access-key', key = 'azuredatabricks-dl-formula1app-tenantId')
    client_secret = dbutils.secrets.get(scope = 'formula1-datalake-access-key', key = 'azuredatabricks-dl-formula1app-client-secret')

    # set spark configurations
    configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": clientID,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenantId}/oauth2/token"}
    
    # unmount the mount point if already exists
    if any(mount.mountPoint == f"/mnt/{storage_account_name}/{container_name}" for mount in dbutils.fs.mounts()):
          dbutils.fs.unmount(f"/mnt/{storage_account_name}/{container_name}")

    #  mount starage account containers
    dbutils.fs.mount(
                    source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/",
                    mount_point = f"/mnt/{storage_account_name}/{container_name}",
                    extra_configs = configs)
    
    display(dbutils.fs.mounts())

In [0]:
mount_adls('dldatabrickscoursedev001', 'presentation')


In [0]:
circuits_df = spark.read.parquet('/mnt/dldatabrickscoursedev001/processed/circuits')
races_df = spark.read.parquet('/mnt/dldatabrickscoursedev001/processed/races')
constructors_df = spark.read.parquet('/mnt/dldatabrickscoursedev001/processed/constructors')
results_df = spark.read.parquet('/mnt/dldatabrickscoursedev001/processed/results')
drivers_df = spark.read.parquet('/mnt/dldatabrickscoursedev001/processed/drivers')

In [0]:
races_sel_df = races_df.withColumnRenamed("name","race_name") \
                   .withColumnRenamed("year","race_year") \
                   .withColumnRenamed("race_timestamp","race_date")

In [0]:
circuits_sel_df = circuits_df.withColumnRenamed("location","circuit_location")

In [0]:
drivers_sel_df = drivers_df.withColumnRenamed("name","driver_name") \
                       .withColumnRenamed("number","driver_number") \
                       .withColumnRenamed("nationality","driver_nationality")

In [0]:
results_df = results_df.withColumnRenamed("time","race_time") 

In [0]:
from pyspark.sql.functions import col, lit, current_timestamp, concat

In [0]:
%run "../includes/configuration"

In [0]:
races_circuits_df = races_sel_df.join(circuits_sel_df, races_sel_df.circuit_id == circuits_sel_df.circuit_id, 'inner') \
.select("race_id","race_name","race_year","race_date","circuit_location")

In [0]:
races_results_df = results_df.join(drivers_sel_df, drivers_sel_df.driver_id == results_df.driver_id, 'inner') \
                             .join(races_circuits_df, races_circuits_df.race_id ==results_df.race_id, 'inner') \
                             .join(constructors_df, constructors_df.constructor_id == results_df.constructor_id, 'inner') \
.select("race_year","race_name","race_date","circuit_location","driver_name","driver_number","driver_nationality",constructors_df.name,"grid","fastest_lap_time","race_time","points")

In [0]:
from pyspark.sql.functions import current_timestamp

In [0]:
races_results_df = races_results_df.withColumn("created_date", current_timestamp()) \
    .withColumnRenamed("name","team")

In [0]:
display(races_results_df)

In [0]:
# display(races_results_df.filter("race_year = 2020 AND race_name == 'Abu Dhabi Grand Prix'"))

In [0]:
races_results_df.write \
    .mode("overwrite")  \
    .parquet(f"{presentation_folder_path}/race_results")